In [1]:
import pandas as pd
import numpy as np
import re
import string
from textblob import TextBlob

In [2]:
from nltk.stem.snowball import SnowballStemmer

In [3]:
stemmer = SnowballStemmer("english")

In [4]:
pd.set_option('display.max_colwidth', -1)

C:\Users\Carretero\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [5]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [6]:
test = pd.read_csv("test.csv",encoding = "ISO-8859-1")
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [7]:
train['cantidad_links'] = train['text'].str.count('http')

In [8]:
test['cantidad_links'] = test['text'].str.count('http')

In [9]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


In [10]:
# Remuevo los links
train['text'] = train['text'].replace(r'http\S+', '', regex=True)
test['text'] = test['text'].replace(r'http\S+', '', regex=True)

## Cuento la cantidad de apariciones de caracteres inusuales

In [11]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
non_alphabetic_characters = ['\?','@','\!','\$']
mean_non_alphabetic_characters = [train[train['text'].str.contains(i)].target.mean() for i in non_alphabetic_characters]
times_non_alphabetic_characters_apears = [len(train[train['text'].str.contains(i)]) for i in non_alphabetic_characters]

In [12]:
print(non_alphabetic_characters)
print(mean_non_alphabetic_characters)
print(times_non_alphabetic_characters_apears)

['\\?', '@', '\\!', '\\$']
[0.27953890489913547, 0.3315350662089259, 0.27298050139275765, 0.3157894736842105]
[1041, 2039, 718, 38]


In [13]:
non_alphabetic_characters = ['?','@','!']

In [14]:
for i in non_alphabetic_characters:
    train[i]=train['text'].apply(lambda x: x.split(' ').count(i))
    test[i]=test['text'].apply(lambda x: x.split(' ').count(i))

In [15]:
# Si tienen un $ seguido de un numero

In [16]:
def money_related(text):
    money_related = False
    for i in range(len(text)-1):
        if(text[i]=='$' and text[i+1].isnumeric()):
            money_related = True
    return(money_related)

In [17]:
train['money_related']=train['text'].apply(lambda x: money_related(x))
test['money_related']=test['text'].apply(lambda x: money_related(x))

In [18]:
#Si tiene una palabra que son todas mayusculas

In [19]:
# Mentions country

In [20]:
# Detectar_fechas,horas,y años

In [21]:
def has_dates(text):
    has_dates = False
    for i in range(len(text)-2):
        if(text[i].isnumeric() and ((text[i+1]=="/") or (text[i+1]=="\\")or  (text[i+1]=="-")) and (text[i+2].isnumeric())):
            has_dates = True
    return(has_dates)

In [22]:
train['tiene_fecha'] = train['text'].apply(lambda x: has_dates(x))
test['tiene_fecha'] = test['text'].apply(lambda x: has_dates(x))

In [23]:
def has_year(text):
    has_year = False
    for i in range(len(text)-4):
        if(text[i]=='2' and text[i+1]=='0' and text[i+2].isnumeric() and text[i+3].isnumeric()):
            has_year = True
        elif(text[i]=='1' and text[i+1]=='9' and text[i+2].isnumeric() and text[i+3].isnumeric()):
            has_year = True
    return(has_year)

In [24]:
train['has_year'] = train['text'].apply(lambda x: has_year(x))
test['has_year'] = test['text'].apply(lambda x: has_year(x))

In [25]:
def otra_forma(text):
    tiene_hora = False
    for i in text:
        if (i.isnumeric()):
            tiene_hora = True
    if(tiene_hora and ( (text.count('PM')>0) or (text.count('AM')>0))   ):
        tiene_hora = True
    else:
        tiene_hora = False
    return(tiene_hora)

In [26]:
def has_time(text):
    has_time = False
    for i in range(len(text)-1):
        if(text[i].isnumeric() and text[i+1]==":"):
            has_time = True
    return(has_time)

In [27]:
train['tiene_hora']=train['text'].apply(lambda x: has_time(x))
test['tiene_hora']=test['text'].apply(lambda x: has_time(x))

In [28]:
train['tiene_hora_2'] = train['text'].apply(lambda x:otra_forma(x))
test['tiene_hora_2'] = test['text'].apply(lambda x:otra_forma(x))

In [29]:
# numeros

In [30]:
def contador_de_numeros(text):
    cantidad = 0
    #primero me aseguro que los numeros no sean fechas u horas 
    
    
    for i in range(0,10):
        if (text.count(str(i))>0):
            cantidad = cantidad+1
    return(cantidad)

In [31]:
#train['numeros'] = train['text'].apply(lambda x: contador_de_numeros(x))

In [32]:
#train['numeros'] = train[(train['numeros']>0) & (train['tiene_fecha']==False) & (train['tiene_fecha_rara']==False) & (train['tiene_hora']==False)].head(50)

# Contador de hashtags

In [33]:
def hashtags_counter(text):
    text = text.lower()
    words = text.split(' ')
    hashtags = []
    for i in words:
        if i.startswith('#'):
            hashtags.append(i)
    return(' '.join(hashtags))

In [34]:
train['hashtags'] = train['text'].apply(lambda x: hashtags_counter(x))
test['hashtags'] = test['text'].apply(lambda x: hashtags_counter(x))

In [35]:
hashtags_por_separado = train.copy()
hashtags_por_separado.set_index(['target'],inplace=True)
hashtags_por_separado = hashtags_por_separado['hashtags'].str.split(expand=True).stack().to_frame()
hashtags_por_separado.columns = ['hashtag']
hashtags_por_separado = hashtags_por_separado.reset_index()
hashtags_por_separado = hashtags_por_separado[['hashtag','target']]
hashtags_por_separado.head()

,hashtag,target
0,#earthquake,1
1,#wildfires,1
2,#alaska,1
3,#wildfires,1
4,#rockyfire,1


In [36]:
hashtag_mas_comunes = hashtags_por_separado.groupby('hashtag').agg({'target':['mean','sum']})
hashtag_mas_comunes.columns = ['promedio_target','cantidad_apariciones']
hashtag_mas_comunes = hashtag_mas_comunes[(hashtag_mas_comunes['promedio_target']>0.6) | (hashtag_mas_comunes['promedio_target']<0.4)]
hashtag_mas_comunes = hashtag_mas_comunes.nlargest(50,'cantidad_apariciones')
hashtag_mas_comunes

,promedio_target,cantidad_apariciones
hashtag,,
#news,0.738462,48
#???,1.000000,23
#hiroshima,1.000000,21
#earthquake,1.000000,19
#??,0.947368,18
#india,1.000000,10
#japan,1.000000,10
#abstorm,1.000000,9
#sismo,0.900000,9


In [37]:
for i in hashtag_mas_comunes.index.values:
    train[i]= train['text'].apply(lambda x: x.count(i))
    test[i]= test['text'].apply(lambda x: x.count(i))

In [38]:
train=train.drop(columns=['hashtags'])
test=test.drop(columns=['hashtags'])

# Relleno keywords nan con hashtags

In [39]:
train_limpio = train.copy()
test_limpio = test.copy()

In [40]:
def devolver_hashtags(text):
    text = text.lower()
    words = text.split(' ')
    result = 0
    hashtags = []
    for i in words:
        if i.startswith('#'):
            hashtags.append(i.split('#')[1])

    if(len(hashtags)==0):
        result='null'
    else:
        result=(' '.join(hashtags))
            
            
    return(result)    

In [41]:
train_limpio.loc[train_limpio['keyword'].isnull(),'keyword'] = train_limpio['text'].apply(lambda x: devolver_hashtags(x))
test_limpio.loc[test_limpio['keyword'].isnull(),'keyword'] = test_limpio['text'].apply(lambda x: devolver_hashtags(x))

## Limpieza de los textos y keywords

In [53]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('target' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text) # quito todo
    return text

In [54]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: clean_text(x))
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))

In [55]:
train_limpio['keyword'] = train_limpio['keyword'].apply(lambda x: str(x).lower())
test_limpio['keyword'] = test_limpio['keyword'].apply(lambda x: str(x).lower())
test_limpio['keyword'] = test_limpio['keyword'].apply(lambda x: clean_text(x))
train_limpio['keyword'] = train_limpio['keyword'].apply(lambda x: clean_text(x))

ANALIZO CUANTOS KEYWORDS CONTIENE CADA TWEET, TENIENDO EN CUENTA TODOS LOS KEYWORDS POSIBLES

In [56]:
dic = list(dict.fromkeys(train_limpio['keyword'].to_list()))
dic.pop(0)
for word in dic:
    index = dic.index(word)
    dic[index] = word.replace('%20',' ')


In [57]:
import re
def suma_de_keywords(lista,string):
    x= 0
    for word in lista:
        x += 1 if (word in string.lower()) else 0
    return x


In [58]:
train_limpio['cant_keywords'] = (train_limpio['text'].apply(lambda x: x.lower())).apply(lambda x: suma_de_keywords(dic,x))
test_limpio['cant_keywords'] = (test_limpio['text'].apply(lambda x: x.lower())).apply(lambda x: suma_de_keywords(dic,x))



In [59]:
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,repeticiones,cant_keywords
0,1,earthquak,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,2
1,4,null,NaN,forest fire near la ronge sask canada,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,3
2,5,null,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,3
3,6,wildfir,NaN,people receive wildfires evacuation orders in california,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,5
4,7,alaska wildfir,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,5


In [60]:
test_limpio.head()

,id,keyword,location,text,cantidad_links,?,@,!,money_related,tiene_fecha,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,repeticiones,cant_keywords
0,0,null,NaN,just happened a terrible car crash,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,1,3
1,2,earthquak,NaN,heard about earthquake is different cities stay safe everyone,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,1,2
2,3,null,NaN,there is a forest fire at spot pond geese are fleeing across the street i cannot save them all,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,1,3
3,9,spokan wildfir,NaN,apocalypse lighting spokane wildfires,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,1,5
4,11,null,NaN,typhoon soudelor kills in china and taiwan,0,0,0,0,False,False,...,0,0,0,0,0,0,0,0,1,3


## Analizar las repeticiones de un tweet

In [61]:
test_limpio['repeticiones'] = test_limpio.groupby('text')['id'].transform('count')
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')

In [62]:
train_limpio.nlargest(20,'repeticiones')

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,repeticiones,cant_keywords
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5881,8400,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5884,8404,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5885,8405,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5888,8409,sandstorm,United States,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4
5889,8410,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,0,0,False,...,0,0,0,0,0,0,0,0,24,4


## Voy a dejar los keywords con stemming

### Con esto busco que los keywords me ayuden a decidir el target pero solo lo hagan si hay "suficientes" como para hacerlo.

In [63]:
rowsWithNonNullKeyword = train_limpio.loc[train_limpio.keyword.notnull()]
rowsWithNonNullKeyword[:4]

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,#newyork,#phoenix,#rockyfire,#sandiego,#somalia,#strategicpatience,#terrorism,#terrorist,repeticiones,cant_keywords
0,1,earthquak,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,2
1,4,null,NaN,forest fire near la ronge sask canada,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,3
2,5,null,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,3
3,6,wildfir,NaN,people receive wildfires evacuation orders in california,1,0,0,0,0,False,...,0,0,0,0,0,0,0,0,1,5


In [64]:
keywordsCount = rowsWithNonNullKeyword.groupby("keyword").agg({"id": "count"}).reset_index()
keywordsCount.columns = ["keyword", "count"]
keywordsCount.head(2)

,keyword,count
0,,1165
1,,1


In [65]:
train_limpio['keyword'] = train_limpio['keyword'].replace({ np.nan:'null'})
train_limpio["keyword"] = train_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [66]:
test_limpio['keyword'] = test_limpio['keyword'].replace({ np.nan:'null'})
test_limpio["keyword"] = test_limpio['keyword'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

In [67]:
keywords = train_limpio.groupby('keyword').agg({'target':['count','mean']})
keywords.columns=['cantidad','target_mean']
keywords

,cantidad,target_mean
keyword,,
,1165,0.583691
,1,1.000000
ablaz,36,0.361111
accid,35,0.685714
aftershock,34,0.000000
...,...,...
windstorm,40,0.400000
worldnew,1,1.000000
wound,70,0.514286


In [68]:
keywords[keywords['cantidad']>20]

,cantidad,target_mean
keyword,,
,1165,0.583691
ablaz,36,0.361111
accid,35,0.685714
aftershock,34,0.000000
ambul,38,0.526316
...,...,...
wildfir,34,0.882353
windstorm,40,0.400000
wound,70,0.514286


In [69]:
# Me armo una columna por cada keyword, no esta bien pero bueno

In [70]:
for i in keywords.index:
    train_limpio[i+'-'+'keyword'] = train_limpio['keyword'].str.contains(i)
    test_limpio[i+'-'+'keyword'] = test_limpio['keyword'].str.contains(i)
# Tengo que agregar el +'-'+'keyword' o sino algunas palabras pueden ser sobreescritas al vectorizar los textos

## Sabemos que hay locations repetidas pero con distinto nombre, veo si puedo agrupar

In [71]:
rowsWithNonNullLocation = train_limpio.loc[train_limpio.location.notnull()].copy()
rowsWithNonNullLocation

,id,keyword,location,text,target,cantidad_links,?,@,!,money_related,...,upheav-keyword,volcano-keyword,weapon-keyword,whirlwind-keyword,wildfir-keyword,windstorm-keyword,worldnew-keyword,wound-keyword,wreck-keyword,wreckag-keyword
31,48,ablaz,Birmingham,bbcmtd wholesale markets ablaze,1,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
32,49,ablaz,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
33,50,ablaz,AFRICA,africanbaze breaking newsnigeria flag set ablaze in aba,1,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
34,52,ablaz,"Philadelphia, PA",crying out for more set me ablaze,0,0,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
35,53,ablaz,"London, UK",on plus side look at the sky last night it was ablaze,0,1,0,0,0,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7575,10826,wreck,TN,on the bright side i wrecked,0,1,0,0,0,False,...,False,False,False,False,False,False,False,False,True,False
7577,10829,wreck,#NewcastleuponTyne #UK,hes gone you can relax i thought the wife who wrecked her cake was a goner mind lol whoops,0,0,0,0,0,False,...,False,False,False,False,False,False,False,False,True,False
7579,10831,wreck,"Vancouver, Canada",three days off from work and theyve pretty much all been wrecked hahaha shoutout to my family for that one,0,0,0,0,0,False,...,False,False,False,False,False,False,False,False,True,False
7580,10832,wreck,London,fx forex trading cramer igers words that wrecked disneys stock,0,1,0,0,0,False,...,False,False,False,False,False,False,False,False,True,False


In [520]:
locationCount = rowsWithNonNullLocation.groupby("location").agg({"id":"count"}).reset_index()
locationCount.columns = ["location", "count"]
locationCount.loc[locationCount["count"] > 4]

,location,count
27,Road to the Billionaires Club,6
114,304,9
175,??????,6
324,"Asheville, NC",5
339,Atlanta,6
...,...,...
2866,WorldWide,5
2868,Worldwide,19
3054,in the Word of God,5
3222,ss,10


### Casos como New York, new york, ny y nyc nos pueden molestar, veo si hay manera de arreglarlo

In [521]:
#list(locationCount.loc[locationCount["count"] > 4]["location"])

#Asheville, NC | Atlanta | Atlanta, GA | Austin, TX | Australia | Boston, MA | Brasil | Brooklyn, NY | Calgary, AB | Calgary, Alberta
#California | California, USA | California, United States | Canada | Charlotte, NC | Chicago | Chicago, IL | Cleveland, OH
#Colorado | Coventry | Dallas, TX | Denver, CO | Denver, Colorado | Earth | Everywhere | Florida | Florida, USA
#Global | Houston, TX | India | Indiana | Indonesia | Ireland | Kenya | Lagos | Lagos, Nigeria | Leeds, England
#London | London, England | London, UK | Los Angeles | Los Angeles, CA | Manchester | Massachusetts
#Melbourne, Australia | Memphis, TN | Morioh, Japan | Mumbai | NY | NYC | Nashville, TN | New Hampshire
#New Jersey | New York | New York City | New York, NY | New York, USA | Newcastle | Nigeria | North Carolina
#Oklahoma City, OK | Orland, FL | Pakistan | Paterson, New Jersey | Pedophile hunting ground | Pennsylvania
#Pennsylvania, USA | Philippines | Planet Earth | Portland, OR | Puerto Rico | Sacramento, CA | San Diego, CA
# San Francisco | San Francisco, CA | San Jose, CA | Scotland | Seattle | Seattle, WA | Singapore | South Africa
# Southern California | Sydney | Tampa, FL | Tennessee | Texas | Texas, USA | The Netherlands | Toronto | UK
# US | USA | United Kingdom | United States | Vancouver, BC | Washington, D.C. | Washington, DC | Winsconsin
# World | WorldWide | Worldwide | in the Word of God | ss | worldwide

In [522]:
realPlaces = {"NC": "NC", "Atlanta": "Atlanta", "TX": "TX", "Texas": "TX", "Australia": "Australia", "Boston": "MA", "MA": "MA", "Brasil": "Brasil", "Brooklyn": "NY", "Calgary": "AB", "AB": "AB", "California": "California", "Canada": "Canada", "Chicago": "Chicago", "Cleveland": "Cleveland", "Colorado": "Colorado", "Coventry": "Coventry", "Dallas": "TX", "Denver": "CO", "CO": "CO", "Colorado": "CO", "Florida": "Florida", "Houston": "TX", "India": "India", "Indiana": "Indiana", "Indonesia": "Indonesia", "Ireland": "Ireland", "Kenya": "Kenya", "Lagos": "Lagos", "Leeds": "UK", "London": "UK", "UK": "UK", "Los Angeles": "Los Angeles", "Manchester": "Manchester", "Massachusetts": "Massachusetts", "TN": "TN", "Japan": "Japan", "Mumbai": "Mumbai", "NY": "NY", "NYC": "NYC", "New Hampshire": "New Hampshire", "New Jersey": "New Jersey", "New York": "NY", "New York City": "NYC", "USA": "USA", "Newcastle": "Newcastle", "Nigeria": "Nigeria", "North Carolina": "NC", "Oklahoma City": "Oklahoma City", "Orland": "Orland", "Pakistan": "Pakistan", "Pennsylvania": "Pennsylvania", "Philippines": "Philippines", "Portland": "Portland", "Puerto Rico": "Puerto Rico", "Sacramento": "Sacramento", "San Diego": "San Diego", "San Francisco": "San Francisco", "San Jose": "San Jose", "Scotland": "Scotland", "Seattle": "Seattle", "Singapore": "Singapore", "South Africa": "South Africa", "Southern California": "Southern California", "Sydney": "Sydney", "Tampa": "Tampa", "Tennesse": "Tennesse", "Texas": "TX", "The Netherlands": "The Netherlands", "Toronto": "Toronto", "UK": "UK", "US": "USA", "USA": "USA", "United Kingdom": "UK", "United States": "USA", "Vancouver": "Vancouver", "Washington": "Washington", "Winsconsin": "Winsconsin"}                                                                                                                                 
realPlacesWithSpaces = ["Los Angeles", "New Hampshire", "New Jersey", "New York", "North Carolina", "Oklahoma City", "Puerto Rico", "San Diego", "San Francisco", "San Jose", "South Africa", "Southern California", "The Netherlands", "United Kingdom", "United States"]
notRealPlaces = ["Earth", "Everywhere", "Global", "Pedophile hunting ground", "Planet Earth", "World", "WorldWide", "Worldwide", "in the Word of God", "ss", "worldwide"]

In [523]:
def getLocationGroup(location):
    if location in notRealPlaces:
        return location
    else:
        for place in realPlacesWithSpaces:
            if(place in location):
                return realPlaces[place]
        locationString = location.replace(",", "")
        for word in locationString.split(" "):
            if word in realPlaces:
                return realPlaces[word]
        return re.escape(location)
    
def isRealPlace(location):
    return getLocationGroup(location) in realPlaces

train_limpio["location"] = train_limpio["location"].replace({ np.nan:'null'})
train_limpio["location"] = train_limpio["location"].map(lambda x: getLocationGroup(x))
train_limpio["isRealPlace"] = train_limpio["location"].map(lambda x: isRealPlace(x))

test_limpio["location"] = test_limpio["location"].replace({ np.nan:'null'})
test_limpio["location"] = test_limpio["location"].map(lambda x: getLocationGroup(x))
test_limpio["isRealPlace"] = test_limpio["location"].map(lambda x: isRealPlace(x))

In [524]:
locations = train_limpio.groupby('location').agg({'target':['count','mean']})
locations.columns=['cantidad','target_mean']
locations

,cantidad,target_mean
location,,
'Merica,1,0.0
'SAN\ ANTONIOOOOO',1,0.0
'soooota,1,1.0
//RP\\\ ot\ @Mort3mer\\\\,1,0.0
//\?\?//,1,1.0
...,...,...
Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?,1,0.0
Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢,1,0.0
Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢,1,0.0


In [525]:
for i in locations.index:
    train_limpio[i+'-'+'location'] = train_limpio['location'].str.contains(i)
    test_limpio[i+'-'+'location'] = test_limpio['location'].str.contains(i)

# Remuevo informacion
## Remuevo el ID

In [526]:
train_limpio.shape

(7613, 2838)

In [527]:
train_limpio = train_limpio.drop(columns=['id'])
test_limpio = test_limpio.drop(columns=['id'])

In [528]:
#train_keywords = train_limpio.pop("keyword")
train_locations = train_limpio.pop("location")
#test_keywords = test_limpio.pop("keyword")
test_locations = test_limpio.pop("location")

## Remuevo los datos repetidos
### No remuevo los repetivos del test obviamente
# Al final los dejo, da un poco mejor conservando los datos repetidos

In [529]:
train_limpio.shape

(7613, 2836)

In [530]:
test_limpio.shape

(3263, 2835)

In [531]:
#train_limpio = train_limpio.drop_duplicates()
#train_limpio.shape

In [532]:
train_limpio.head()

,keyword,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,...,"ÃÃT:\ 43\.631838,\-79\.55807-location","ÃÃT:\ 6\.4682,3\.18287-location","ÃÃT:\ 6\.488400524109015,3\.352798039832285-location","ÃÃT:\ \-26\.695807,27\.837865-location","ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location
0,earthquak,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,null,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,null,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,wildfir,people receive wildfires evacuation orders in california,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,alaska wildfir,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Longitud de los tweets

In [533]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [534]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [535]:
train_limpio.head(2)

,keyword,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,...,"ÃÃT:\ 6\.4682,3\.18287-location","ÃÃT:\ 6\.488400524109015,3\.352798039832285-location","ÃÃT:\ \-26\.695807,27\.837865-location","ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location,longitud_tweet
0,earthquak,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,70
1,null,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,40


In [536]:
test_limpio['longitud_tweet'] = test_limpio['text'].apply(lambda x: roundup(len(x)))

## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [537]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [538]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<=2 and max_repeated_letters<=2):
            new_text.append(word)  #remuevo las palabras que tengan 3 o mas letras seguidas repetidas
    return ( ' '.join(new_text))

In [539]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

test_limpio['letras_seguidas'] = test_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [540]:
letras_seguidas_train = train_limpio['letras_seguidas']
letras_seguidas_test = test_limpio['letras_seguidas']

In [541]:
#Remuevo esas palabras, x ahora las dejo
train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
test_limpio['text'] = test_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
train_limpio.head(2)

,keyword,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,...,"ÃÃT:\ 6\.488400524109015,3\.352798039832285-location","ÃÃT:\ \-26\.695807,27\.837865-location","ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location,longitud_tweet,letras_seguidas
0,earthquak,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,70,2
1,null,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,40,1


### Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

# Correccion de typos

### Lo deje comentado por ser demasiado lento

In [542]:
#Prueba basica
#pruebita = train_limpio.iloc[1:150].copy()

def spell_correcter(text):
    word_list = text.split(' ')
    return [str(TextBlob(word).correct()) for word in word_list]

from textblob import TextBlob
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

#pruebita['text_2'] = pruebita['text'].apply(lambda x: ' '.join(x))
#pruebita.head()

In [543]:
#Confirmé que funciona, lo aplico a todas

In [544]:
#train_limpio['text'] = train_limpio['text'].apply(lambda x: spell_correcter(x))

#train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: spell_correcter(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: ' '.join(x))

In [545]:
#train_limpio.to_csv('train_con_textos_corregidos.csv', index=False)

#test_limpio.to_csv('test_con_textos_corregidos.csv', index=False)

In [546]:
#train_texto_corregido = pd.read_csv("train_con_textos_corregidos.csv",encoding = "ISO-8859-1")
#test_texto_corregido = pd.read_csv("test_con_textos_corregidos.csv",encoding = "ISO-8859-1")

In [547]:
#train_texto_corregido = train_texto_corregido.text.fillna(0)

In [548]:
#test_texto_corregido = test_texto_corregido.text.fillna(0)

In [549]:
#train_limpio['texto_corregido'] = train_texto_corregido

In [550]:
#test_limpio['texto_corregido'] = test_texto_corregido

In [551]:
#train_limpio[train_limpio['texto_corregido']!=train_limpio['text']].head()

In [552]:
#Corrige typos que no deberia, mejor no lo uso

# Analisis de sentimiento y de objetividad

In [553]:
b = TextBlob("Simple is better than complex")
b.sentiment

Sentiment(polarity=0.06666666666666667, subjectivity=0.41904761904761906)

In [554]:
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))
pruebita = train_limpio.iloc[1:150].copy()
pruebita['analisis_sentimientos'] = pruebita['text'].apply(lambda x: (TextBlob(str(x)).sentiment))
pruebita.head()

,keyword,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,...,"ÃÃT:\ \-26\.695807,27\.837865-location","ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location,longitud_tweet,letras_seguidas,analisis_sentimientos
1,null,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,40,1,"(0.1, 0.4)"
2,null,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,130,2,"(-0.018750000000000003, 0.3875)"
3,wildfir,people receive wildfires evacuation orders in california,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,60,1,"(0.0, 0.0)"
4,alaska wildfir,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,90,2,"(0.0, 0.0)"
5,rockyfir cafir wildfir,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,100,1,"(-0.1125, 0.2375)"


In [555]:
train_limpio['sentimiento'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
train_limpio['objetividad'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [556]:
test_limpio['sentimiento'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
test_limpio['objetividad'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [557]:
train_limpio.to_csv('train_limpio_sin_vect_textos.csv', index=False)
test_limpio.to_csv('test_limpio_sin_vect_textos.csv', index=False)

# Stemming

In [101]:
test_limpio.shape

(3263, 2852)

In [102]:
train_limpio.shape

(7613, 2852)

In [421]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [422]:
import string

def remove_punctuation(text):
    '''a function for removing punctuation'''
    # replacing the punctuations with no space.
    # which in effect deletes the punctuation marks.
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks.
    return text.translate(translator)



In [423]:
test_limpio['text'] = test_limpio['text'].apply(remove_punctuation)
train_limpio['text'] = train_limpio['text'].apply(remove_punctuation)

In [424]:
# create an object of stemming function.
stemmer = SnowballStemmer("english")

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 



In [425]:
train_limpio['text'] = train_limpio['text'].apply(stemming)
test_limpio['text'] = test_limpio['text'].apply(stemming)

In [104]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word',max_features=5000 ,binary=True)
vectorizer.fit(train_limpio['text'])
new_train = vectorizer.transform(train_limpio['text']).todense()
train_limpio.head(1)

,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,tiene_hora,...,"ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,Our Deeds are the Reason of this earthquake May ALLAH Forgive us all,1,0,0,0,0,False,False,False,False,...,False,False,False,False,False,False,70,2,0.0,0.0


In [105]:
new_train = pd.DataFrame(new_train, columns=vectorizer.get_feature_names())

In [106]:
train_text = train_limpio.pop('text')

In [107]:
train_limpio = train_limpio.reset_index(drop=True)

In [108]:
for i in train_limpio.columns:
    new_train[i]=train_limpio[i]

In [109]:
#Ahora lo hago con test
new_test = vectorizer.transform(test_limpio['text']).todense()
new_test = pd.DataFrame(new_test, columns=vectorizer.get_feature_names())

In [110]:
test_text = test_limpio.pop('text')

In [111]:
test_limpio = test_limpio.reset_index(drop=True)

In [112]:
for i in test_limpio.columns:
    new_test[i]=test_limpio[i]

In [113]:
new_test = new_test.drop(columns=['target']) #lo agrego porque sino agrega la palabra 'target' por el bow

## Guardo los csvs limpios para mas adelante

In [114]:
new_train.to_csv('train_limpio_con_BOW_de_5000_y_Stemming.csv', index=False)

In [115]:
new_test.to_csv('test_limpio_con_BOW_de_5000_y_Stemming.csv', index=False)

In [438]:
new_train_withNoDroppedColumns = new_train
new_test_withNoDroppedColumns = new_test

new_train_withNoDroppedColumns["text"] = train_text
new_test_withNoDroppedColumns["text"] = test_text

new_train_withNoDroppedColumns["keyword"] = train_keywords
new_train_withNoDroppedColumns["location"] = train_locations
new_test_withNoDroppedColumns["keyword"] = test_keywords
new_test_withNoDroppedColumns["location"] = test_locations

In [439]:
new_train_withNoDroppedColumns.to_csv('train_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv', index=False)
new_test_withNoDroppedColumns.to_csv('test_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv', index=False)

In [440]:
test_limpio["text"] = test_text
train_limpio["text"] = train_text
train_limpio.to_csv('train_limpio_input_2_NN.csv', index=False)
test_limpio.to_csv('test_limpio_input_2_NN.csv', index=False)